In [1]:
!pip install pypdf sentence-transformers faiss-cpu huggingface_hub --quiet


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.4 which is incompatible.


In [2]:
!pip install -q streamlit

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
import faiss
from huggingface_hub import InferenceClient

import numpy as np
import textwrap
import os
import streamlit as st

Disabling PyTorch because PyTorch >= 2.1 is required but found 1.12.1


ImportError: Traceback (most recent call last):
  File "C:\Users\John Elkess\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
#Load your PDF
pdf_path = "/kaggle/input/cv-ass2/Assignment 2 2025.pdf"

reader = PdfReader(pdf_path)
all_text = ""

for page in reader.pages:
    text = page.extract_text()
    if text:
        all_text += text + "\n"

# Preview the start of the text
print("Extracted text preview:\n")
print(all_text[:500])


Extracted text preview:

German University in Cairo 
Faculty of Media Engineering and Technology 
DMET 901 – Computer Vision 
Assignment #2 
              Due on Thursday, 4th of December @ 11:59PM  
 
The main aim of this assignment is to analyze the interaction between a corner detection scoring 
function (SUSAN) and automatic threshold detection (Convex Hull). As per that, the following 
components are to be implemented: 
1. Corner detection: 
a. SUSAN. 
2. Automatic threshold detection: 
a. Convex hull. 
 
Corner De


In [ ]:
#Chunking the text
def chunk_text(text, max_length=400, overlap=100):
    chunks = []
    current = []

    for line in text.split("\n"):
        line = line.strip()
        if not line:
            continue

        # If adding the new line exceeds max_length → finalize chunk
        if sum(len(x) for x in current) + len(line) > max_length:
            chunk = " ".join(current)
            chunks.append(chunk)

            # --- NEW: create overlap ---
            # Convert chunk into characters and take the last `overlap` chars
            if overlap > 0:
                overlap_text = chunk[-overlap:]
                current = [overlap_text, line]  # start new chunk with overlap + new line
            else:
                current = [line]

        else:
            current.append(line)

    # Add the final chunk
    if current:
        chunks.append(" ".join(current))

    return chunks


chunks = chunk_text(all_text)
print(f"Total chunks created: {len(chunks)}")
print("Sample chunk:\n", chunks[0][:300], "...")

Total chunks created: 17
Sample chunk:
 German University in Cairo Faculty of Media Engineering and Technology DMET 901 – Computer Vision Assignment #2 Due on Thursday, 4th of December @ 11:59PM The main aim of this assignment is to analyze the interaction between a corner detection scoring function (SUSAN) and automatic threshold detecti ...


In [ ]:
#Load embedding model
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

#Encode chunks
print("Encoding chunks... this may take ~10–20 seconds.")
embeddings = embedder.encode(chunks, convert_to_numpy=True)

embeddings.shape


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Encoding chunks... this may take ~10–20 seconds.


(17, 384)

In [ ]:
#Build FAISS Index
d = embeddings.shape[1] 
index = faiss.IndexFlatL2(d)

# Add vectors to index
index.add(embeddings)

print("FAISS index built successfully!")
print("Number of vectors in the index:", index.ntotal)


FAISS index built successfully!
Number of vectors in the index: 17


In [ ]:
#Retrieval function
def retrieve(query, k=3):
    # Embed the user query using the same SentenceTransformer model used for document chunks
    query_emb = embedder.encode([query], convert_to_numpy=True)
    
    # Search for the query using the FAISS index of our chunks
    distances, indices = index.search(query_emb, k)

    print(distances, indices)
    print("**Meaning: Best match is chunk", indices[0][0], "with distance", distances[0][0], ",and Second best is chunk", indices[0][1], "with distance ", distances[0][1], "**")
    
    # Return the retrieved chunks
    results = [chunks[i] for i in indices[0]]
    return results

# Test retrieval
sample_query = "What is the main topic discussed in the document?"
retrieved_chunks = retrieve(sample_query, k=2)


print("Top retrieved chunks:\n")
for i, c in enumerate(retrieved_chunks, 1):
    print(f"---- Chunk {i} ----")
    print(c[:400], "\n")


[[1.6624113 1.7481887]] [[16  7]]
**Meaning: Best match is chunk 16 with distance 1.6624113 ,and Second best is chunk 7 with distance  1.7481887 **
Top retrieved chunks:

---- Chunk 1 ----
d will be a ZERO. • You are not allowed to use any predefined functions for any of the requirements. •  The deadline to submit the assignment is on Thursday, 4th of December, 2025 at 11:59 PM 

---- Chunk 2 ----
r splitting it into three regions which are: 1. lower region which is the image below the threshold. 2. mid region which is the image between the 2 thresholds. 3. higher region which is the image above the higher threshold. German University in Cairo Faculty of Media Engineering and Technology DMET 901 – Computer Vision Assignment #2 Due on Thursday, 4th of December @ 11:59PM Test Scenario: 



In [ ]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import InferenceClient

user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("John") #"RAG-Lab8" is your Secret Label


client = InferenceClient(
    model="google/gemma-2-2b-it",
    token=HF_TOKEN
)

def call_llm(prompt, max_tokens=256):
    response = client.chat_completion(
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=0.2, #temperature: controls randomness of the provided answer
    )
    return response.choices[0].message["content"]


In [ ]:
def rag_answer(query, k=3, max_tokens=500):
    # Step 1: Retrieve relevant chunks
    retrieved = retrieve(query, k=k)

    # Step 2: Build the context
    context = "\n\n".join(retrieved)

    # Step 3: Build prompt for the LLM
    prompt = f"""
You are a helpful assistant using Retrieval-Augmented Generation.

Here is relevant context extracted from a document:

{context}

Based on this context, answer the following question:

Question: {query}

If the answer is not in the context, say: "The document does not contain the answer."
"""

    # Step 4: Call Gemma 2B Instruct (conversational mode)
    response = client.chat_completion(
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=0.2,
    )

    return response.choices[0].message["content"]

In [ ]:
print(rag_answer("What should be included in report Milestone 1?"))


[[1.4527651 1.5948305 1.6682742]] [[16  7 10]]
**Meaning: Best match is chunk 16 with distance 1.4527651 ,and Second best is chunk 7 with distance  1.5948305 **
The document does not contain the answer. 



# RAG Using LangChain

In [ ]:
!pip install langchain langchain-community langchain-huggingface --quiet


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.8/450.8 kB 21.4 MB/s eta 0:00:00


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS as LCFAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_huggingface import ChatHuggingFace
from langchain.llms.base import LLM
from typing import Optional, List, Any
from pydantic import BaseModel, Field

In [ ]:
# Custom LLM wrapper for HuggingFace Inference Client (Gemma conversational)
class GemmaLangChainWrapper(LLM):
    client: Any = Field(...)
    max_tokens: int = 500 #sets a default max output length

    @property
    def _llm_type(self) -> str:
        return "gemma_hf_api" #Identify the LLM type

    #what LangChain calls when it needs the LLM to answer something
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = self.client.chat_completion( #call the HuggingFace API
            messages=[{"role": "user", "content": prompt}],  #Wrap the plain text prompt into chat format because Gemma ONLY understands chat messages.
            max_tokens=self.max_tokens,
            temperature=0.2,
        )
        return response.choices[0].message["content"]


# Instantiate the wrapper
gemma_llm = GemmaLangChainWrapper(client=client)


In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=100
)
documents = splitter.create_documents([all_text])
len(documents)

17

In [ ]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

/tmp/ipykernel_47/2127729888.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [ ]:
vectorstore = LCFAISS.from_documents(
    documents=documents,
    embedding=embedding_model
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=gemma_llm,
    retriever=retriever,
    chain_type="stuff" #concatenate all retrieved documents and feed them to the LLM as one big prompt.
)

In [ ]:
response = qa_chain.run("What should be included in the CV assignment")
print(response)

/tmp/ipykernel_47/3134864812.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run("What should be included in the CV assignment")


The assignment requires you to implement a corner detection scoring function (SUSAN) and use it to analyze an image. You need to:

1. **Implement the SUSAN scoring function:** This means writing code that calculates the SUSAN score for each pixel in an image.
2. **Use automated threshold detection:** You need to implement a convex hull algorithm to determine two threshold values.
3. **Split the image into three regions:** Based on the two threshold values, divide the image into three regions:
    * **Lower region:** Pixels below the lower threshold.
    * **Mid region:** Pixels between the two thresholds.
    * **Higher region:** Pixels above the higher threshold.
4. **Submit your code:** You need to submit a notebook (.ipynb/.py) containing your code for all the steps mentioned above.

**Important Notes:**

* **No copying code:** You are not allowed to copy code from other teams or ChatGPT.
* **No predefined functions:** You cannot use any predefined functions for the requirements.
* 

In [ ]:
@st.cache_resource(show_spinner="Building index from Checklist.pdf...")
def build_vectorstore():
    # Update path if your PDF has a different name/location
    pdf_path = '/kaggle/input/ms1-checklist/Milestone 1 checklist.pdf'  # or "data/Checklist.pdf" etc.

    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        t = page.extract_text()
        if t:
            text += t + "\n"

    # Chunking – same as lab
    splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100)
    docs = splitter.create_documents([text])

    # Embeddings – same as lab
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # FAISS vector store
    vectorstore = LCFAISS.from_documents(docs, embeddings)
    return vectorstore

# Build once
vectorstore = build_vectorstore()
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
print("Vector store ready!")

2025-12-07 20:17:39.731 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:17:39.733 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:17:39.734 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:17:39.736 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:17:40.238 Thread 'Thread-9': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:17:40.239 Thread 'Thread-9': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:17:40.240 Thread 'Thread-9': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:17:41.749 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode

Vector store ready!


In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("John")

if not hf_token or not hf_token.startswith("hf_"):
    st.warning("Please enter a valid Hugging Face token")
    st.stop()

client = InferenceClient(model="google/gemma-2-2b-it", token=hf_token)
gemma_llm = GemmaLangChainWrapper(client=client)
st.success("Gemma-2-2b-it connected!")

2025-12-07 20:20:49.737 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:20:49.743 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:20:49.748 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=gemma_llm,
    chain_type="stuff",                    # ← required
    retriever=retriever,
    return_source_documents=True           # ← to show sources (encouraged)
)

st.success("RAG chain ready!")

2025-12-07 20:20:08.230 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:20:08.232 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:20:08.233 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [ ]:
result = qa_chain.invoke({"query": "What should be included in theCV assignment"})

st.write("**Answer:**")
st.write(result["result"])

with st.expander("Show source chunks"):
    for i, doc in enumerate(result["source_documents"], 1):
        st.caption(f"Source {i}: {doc.page_content.strip()[:500]}...")

2025-12-07 20:20:09.101 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:20:09.102 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:20:09.103 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:20:09.104 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:20:09.105 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:20:09.106 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:20:09.108 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:20:09.108 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
st.set_page_config(page_title="CV RAG Chatbot", page_icon="🤖")

st.title("CV Assignment RAG Chatbot")
st.markdown("Ask about CV Assignment requirements. Answers grounded in the document.")

with st.sidebar:
    st.header("Document")
    st.write("**CV.pdf**")
    if st.button("Rebuild Index"):
        st.cache_resource.clear()
        st.rerun()

query = st.chat_input("e.g., What should be included in report Milestone 1?")
if query:
    with st.spinner("Generating..."):
        result = qa_chain.invoke({"query": query})

    st.subheader("Answer")
    st.write(result["result"])

    with st.expander("View sources", expanded=False):
        for i, doc in enumerate(result["source_documents"], 1):
            st.caption(f"Source {i}: {doc.page_content.strip()}")

st.caption("CSEN 903 Lab 8 | Winter 2025")

2025-12-07 20:20:09.131 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:20:09.134 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:20:09.135 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:20:09.136 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:20:09.137 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:20:09.138 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:20:09.141 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 20:20:09.142 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()